In [1]:
from llm_operator import Relu, Relu_backward
import numpy as np

In [2]:
rng = np.random.default_rng(1000)

In [3]:
epsilon = 1e-6

In [4]:
B = 2
T = 4
C = 6

In [5]:
X = 2 * (rng.random((B, T, C), np.float64) - 0.5)

In [6]:
dY = 2 * (rng.random((B, T, C), np.float64) - 0.5)

In [7]:
X

array([[[ 0.04277148,  0.20768369, -0.05811641, -0.59350411,
          0.05751805, -0.61792744],
        [-0.4369088 ,  0.5073631 ,  0.10334355,  0.72744415,
          0.61074444, -0.50325467],
        [-0.62028518,  0.96799116,  0.33999433, -0.43923434,
         -0.59217353,  0.25012937],
        [ 0.30520863,  0.79761506,  0.94952756, -0.69213526,
          0.39817855, -0.10551711]],

       [[-0.96497358, -0.41795019, -0.23752678, -0.35794418,
          0.88508934,  0.40533395],
        [-0.72709936, -0.31358185,  0.62398921, -0.703012  ,
         -0.88134862, -0.37116673],
        [-0.15968709,  0.61603542, -0.98098483, -0.09183242,
          0.11737397, -0.99422273],
        [-0.40448486, -0.89240178,  0.13533751,  0.8811163 ,
          0.44854743,  0.71275618]]])

In [8]:
Y, relu_cache = Relu(X)

In [9]:
dX_num = np.zeros_like(X)

In [14]:
def grad_check_Activation(activate, activate_backword, eps=1e-6):
    rng = np.random.default_rng(1000)

    B, T, C = 2, 4, 6
    X = 2 * rng.random((B, T, C), dtype=np.float64) - 1
    dY = 2 * rng.random((B, T, C), dtype=np.float64) - 1

    # forward
    Y, cache = activate(X)

    # analytical gradient
    dX = activate_backword(dY, cache)

    # numerical gradient
    dX_num = np.zeros_like(X)
    
    iteration = np.nditer(X, flags=["multi_index"], op_flags=['readwrite'])
    while not iteration.finished:
        idx = iteration.multi_index
        # print(idx)
        old = X[idx]

        X[idx] = old + epsilon
        Yp, _ = activate(X)
        Lp = np.sum(Yp * dY)

        X[idx] = old - epsilon
        Ym, _ = activate(X)
        Lm = np.sum(Ym * dY)

        X[idx] = old
        dX_num[idx] = (Lp - Lm) / (2 * eps)

        iteration.iternext()

    max_abs = np.max(np.abs(dX - dX_num))
    rel = max_abs / (np.max(np.abs(dX) + np.abs(dX_num)) + 1e-22)

    print("max_abs_diff:", max_abs)
    print("relative_diff:", rel)


In [15]:
grad_check_Activation(Relu, Relu_backward, eps=1e-6)

max_abs_diff: 1.9616686053325338e-10
relative_diff: 1.0041134203920265e-10
